In [51]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler


scalar = MinMaxScaler()

X_train = pd.read_csv('trainX.csv')  
y_train = pd.read_csv('trainY.csv')

y_train = y_train.replace({"Over": True, "Under": False})


X_train = X_train.to_numpy(dtype=np.float32)
y_train = y_train.to_numpy(dtype=np.float32)

X_train = scalar.fit_transform(X_train)
# X_train = torch.tensor(X_train, dtype=torch.float32)
# y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1) 


test_X = pd.read_csv('testX.csv')
test_y = pd.read_csv('testY.csv')

test_y = test_y.replace({"Over": 1, "Under": 0})

test_X = test_X.to_numpy(dtype=np.float32)
test_y = test_y.to_numpy(dtype=np.float32)

test_X = scalar.transform(test_X)

def toSequences(seq_size, X, y):
    X_seq = []
    y_seq = []
    for i in range(len(X) - seq_size):
        X_seq.append(X[i:i+seq_size])
        y_seq.append(y[i+seq_size-1]) 
    return torch.tensor(X_seq, dtype=torch.float32).view(-1, seq_size, 19), torch.tensor(y_seq, dtype=torch.float32).view(-1, 1)

X_train, y_train = toSequences(20, X_train, y_train)
X_test, y_test = toSequences(20, test_X, test_y)

C:\Users\Twk23\AppData\Local\Temp\ipykernel_21360\3520892795.py:15: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  y_train = y_train.replace({"Over": True, "Under": False})
C:\Users\Twk23\AppData\Local\Temp\ipykernel_21360\3520892795.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_y = test_y.replace({"Over": 1, "Under": 0})


In [52]:
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

torch.Size([2608, 20, 19]) torch.Size([2608, 1])
torch.Size([855, 20, 19]) torch.Size([855, 1])


In [53]:
# class CNN_LSTM_Model(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size):
#         super(CNN_LSTM_Model, self).__init__()
        
#         # CNN layers
#         self.cnn = nn.Conv1d(in_channels=input_size, 
#                              out_channels=cnn_out_channels, 
#                              kernel_size=kernel_size)
        
#         # LSTM layers
#         self.lstm = nn.LSTM(input_size=cnn_out_channels, 
#                             hidden_size=hidden_size, 
#                             num_layers=num_layers, 
#                             batch_first=True)
        
#         # Fully connected layer for output
#         self.fc = nn.Linear(hidden_size, output_size)

#     def forward(self, x):
#         x = x.permute(0, 2, 1)
#         x = torch.relu(self.cnn(x))
#         x = x.permute(0, 2, 1)
#         x, _ = self.lstm(x)
#         x = x[:, -1, :]
#         x = self.fc(x)
        
#         return x

# input_size = 2608


# sequence_length = 1  
# num_samples = X_train.shape[0]
# num_features = X_train.shape[1]
# num_sequences = num_samples // sequence_length
# X = X_train.reshape(num_sequences, num_features, sequence_length)
# # y = y.view(-1, sequence_length)

# X_data_permuted = X.permute(0, 2, 1) 

# print(X_data_permuted.shape)

# train_dataset = TensorDataset(X_data_permuted, y[:num_sequences])  
# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)


# input_size = num_features 
# hidden_size = 64
# num_layers = 2
# cnn_out_channels = 32
# kernel_size = sequence_length
# output_size = 1  
# learning_rate = 0.001
# num_epochs = 80

# model = CNN_LSTM_Model(input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size)

# criterion = nn.BCEWithLogitsLoss()
# optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# # Training loop
# for epoch in range(num_epochs):
#     model.train()
#     running_loss = 0.0

#     for batch_idx, (inputs, targets) in enumerate(train_loader):
#         optimizer.zero_grad()

#         outputs = model(inputs)
        
#         loss = criterion(outputs.squeeze(), targets.squeeze())  

#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

# print("Training complete!")

In [54]:
# from sklearn.metrics import accuracy_score, confusion_matrix

# sequence_length = 1 
# num_samples = test_X.shape[0]
# num_features = test_X.shape[1]
# num_sequences = num_samples // sequence_length
# test_X = test_X.reshape(num_sequences, num_features, sequence_length) 
# test_X_data_permuted = test_X.permute(0, 2, 1)

# test_dataset = TensorDataset(test_X_data_permuted, test_y[:num_sequences])  # Ensure y matches the number of sequences
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# model.eval()

# all_preds = []
# all_labels = []

# with torch.no_grad():  
#     for inputs, targets in test_loader:

#         outputs = model(inputs)
        
#         targets = targets.squeeze()

#         all_preds.extend(torch.sigmoid(outputs).squeeze().cpu().numpy())  
#         all_labels.extend(targets.cpu().numpy()) 

# print(all_preds)
# all_preds_binary = [1 if pred > 0.5 else 0 for pred in all_preds]

# accuracy = accuracy_score(all_labels, all_preds_binary)
# print(f"Accuracy: {accuracy:.4f}")

# conf_matrix = confusion_matrix(all_labels, all_preds_binary)
# print(f"Confusion Matrix:\n{conf_matrix}")

In [55]:
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size, dropout=0.1):
        super(TransformerTimeSeriesModel, self).__init__()
        
        self.input_fc = nn.Linear(input_size, d_model)
        
        self.positional_encoding = nn.Parameter(
            torch.zeros(1000, d_model)  # Fixed size for simplicity; adjust as needed
        )
        
        # Transformer Encoder
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout),
            num_layers=num_encoder_layers
        )
        
        # Fully connected output layer
        self.output_fc = nn.Linear(d_model, output_size)

        # Optional dropout to avoid overfitting
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, x):
        # Project input to d_model
        x = self.input_fc(x)  # Shape: (batch_size, seq_len, d_model)
        
        # Add positional encoding
        seq_len = x.size(1)
        x = x + self.positional_encoding[:seq_len, :].unsqueeze(0)
        
        # Transpose for Transformer (seq_len, batch_size, d_model)
        x = x.permute(1, 0, 2)
        
        # Apply Transformer Encoder
        x = self.transformer_encoder(x)  # Shape: (seq_len, batch_size, d_model)
        
        # Aggregate outputs (e.g., last time step or mean pooling)
        x = x.mean(dim=0)  # Alternative: x[-1, :, :] for the last time step
        
        # Apply dropout
        x = self.dropout(x)
        
        # Fully connected output layer
        x = self.output_fc(x)
        
        return x

train_dataset = TensorDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

input_size = 19       
d_model = 64           
nhead = 4               
num_encoder_layers = 3  
dim_feedforward = 128    
output_size = 1          

model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size)


# Calculate the imbalanced ratio
num_positive = (y_train == 1).sum().item()
num_negative = (y_train == 0).sum().item()
imbalanced_ratio = num_negative / num_positive

pos_weight = torch.tensor([imbalanced_ratio])
criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
optimizer = optim.Adam(model.parameters(), lr=0.001)

model.train()

# Training loop
num_epochs = 100
for epoch in range(num_epochs):
    running_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()  # Zero gradients

        # Forward pass
        outputs = model(X_batch)
        loss = criterion(outputs.squeeze(), y_batch.squeeze())

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)

    epoch_loss = running_loss / len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

c:\Users\Twk23\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Epoch 1/100, Loss: 0.7162
Epoch 2/100, Loss: 0.7115
Epoch 3/100, Loss: 0.7112
Epoch 4/100, Loss: 0.7079
Epoch 5/100, Loss: 0.7096
Epoch 6/100, Loss: 0.7092
Epoch 7/100, Loss: 0.7111
Epoch 8/100, Loss: 0.6959
Epoch 9/100, Loss: 0.6768
Epoch 10/100, Loss: 0.6615
Epoch 11/100, Loss: 0.6566
Epoch 12/100, Loss: 0.6524
Epoch 13/100, Loss: 0.6440
Epoch 14/100, Loss: 0.6420
Epoch 15/100, Loss: 0.6324
Epoch 16/100, Loss: 0.6249
Epoch 17/100, Loss: 0.6191
Epoch 18/100, Loss: 0.6135
Epoch 19/100, Loss: 0.6063
Epoch 20/100, Loss: 0.6008
Epoch 21/100, Loss: 0.5963
Epoch 22/100, Loss: 0.5974
Epoch 23/100, Loss: 0.5893
Epoch 24/100, Loss: 0.5837
Epoch 25/100, Loss: 0.5751
Epoch 26/100, Loss: 0.5713
Epoch 27/100, Loss: 0.5718
Epoch 28/100, Loss: 0.5495
Epoch 29/100, Loss: 0.5527
Epoch 30/100, Loss: 0.5449
Epoch 31/100, Loss: 0.5477
Epoch 32/100, Loss: 0.5326
Epoch 33/100, Loss: 0.5296
Epoch 34/100, Loss: 0.5228
Epoch 35/100, Loss: 0.5148
Epoch 36/100, Loss: 0.5223
Epoch 37/100, Loss: 0.4926
Epoch 38/1

In [56]:
from sklearn.metrics import accuracy_score, confusion_matrix

model.eval()

with torch.no_grad():
    predictions = torch.round(torch.sigmoid(model(X_test)))

# print(predictions)
predictions = predictions.cpu().numpy().flatten()
accuracy = accuracy_score(predictions, y_test)
print(f"Accuracy: {accuracy:.4f}")

conf_matrix = confusion_matrix(y_test, predictions)
print(f"Confusion Matrix:\n{conf_matrix}")

Accuracy: 0.5860
Confusion Matrix:
[[299 171]
 [183 202]]
